In [ ]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge, Lasso

In [ ]:
file_path = "/content/merged_data_2024_column.csv" # CHANGE TO NEW FILE PATH
df = pd.read_csv(file_path)
print(len(df))

<ipython-input-2-afbfa55173df>:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


668892


In [ ]:
df.drop(columns= ["sellingprice", "zipcode", "sale_year"], inplace= True)

In [ ]:
print(len(df))
missing_summary_df1 = pd.DataFrame({
    "Missing Values": df.isnull().sum(),
    "Missing Percentage (%)": (df.isnull().sum() / len(df)) * 100
})
print(missing_summary_df1)

668892
            Missing Values  Missing Percentage (%)
year                     0                     0.0
make                     0                     0.0
model                    0                     0.0
trim                     0                     0.0
state                    0                     0.0
condition                0                     0.0
odometer                 0                     0.0
color                    0                     0.0
interior                 0                     0.0
mmr                      0                     0.0
2024_price               0                     0.0


#Whole Dataset LightGBM

In [ ]:
categorical_cols = ['make', 'model', 'trim', 'state', 'color', 'interior']
numerical_cols = ['year', 'condition', 'odometer', 'mmr']
cluster_cols = ['year', 'condition', 'odometer', 'mmr']

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder

X = df.drop('2024_price', axis=1)
y = df['2024_price']



label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le

X_numerical = X[numerical_cols].values
X_combined = np.hstack((X[categorical_cols].values, X_numerical))


X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2)


model = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)


from sklearn.metrics import root_mean_squared_error
rmse = root_mean_squared_error(y_test, y_pred)
print(f"Root Mean Squared Error: {rmse}")


from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.053747 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1446
[LightGBM] [Info] Number of data points in the train set: 535113, number of used features: 10
[LightGBM] [Info] Start training from score 17325.880509


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Root Mean Squared Error: 5483.754146773576
R-squared: 0.8439104853320114


#Clustering

In [ ]:
# Only cluster on numeric features and exclude target
numerical_features = df[numerical_cols]

# Scale features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(numerical_features)

In [ ]:
# Apply KMeans clustering
kmeans = KMeans(n_clusters=3, random_state= 42)
kmeans.fit(scaled_features)
cluster_labels = kmeans.labels_
df['cluster'] = cluster_labels

In [ ]:
print(len(df[df["cluster"]==0]))
print(len(df[df["cluster"]==1]))
print(len(df[df["cluster"]==2]))

136622
461080
71190


# Regression on Clusters

In [ ]:
regression_results = {}

for cluster in df["cluster"].unique():
  X = df[df["cluster"] == cluster].drop(['2024_price', "cluster"], axis=1)

  y = df[df["cluster"] == cluster]['2024_price']

  X_numerical = X[numerical_cols]
  scaler = StandardScaler()
  scaled_features = scaler.fit_transform(X_numerical)

  label_encoders = {}
  for col in categorical_cols:
      le = LabelEncoder()
      X[col] = le.fit_transform(X[col].astype(str))
      label_encoders[col] = le



  X_combined = np.hstack((X[categorical_cols].values, scaled_features))
  # print(X_combined[5])

  X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2)

  print(f"-----Training Models for Cluster {cluster}-----")
  ridge_model = Ridge(alpha= 10)
  lasso_model = Lasso()

  ridge_model.fit(X_train, y_train)
  lasso_model.fit(X_train, y_train)

  ridge_y_pred = ridge_model.predict(X_test)
  lasso_y_pred = lasso_model.predict(X_test)

  ridge_rmse = root_mean_squared_error(y_test, ridge_y_pred)
  lasso_rmse = root_mean_squared_error(y_test, lasso_y_pred)

  ridge_r2 = r2_score(y_test, ridge_y_pred)
  lasso_r2 = r2_score(y_test, lasso_y_pred)

  regression_results[cluster] = {
      "Ridge": {"model": ridge_model, "RMSE": ridge_rmse, "R2": ridge_r2},
      "Lasso": {"model": lasso_model, "RMSE": lasso_rmse, "R2": lasso_r2}
  }
  print(f"Training Complete for Cluster {cluster}\n\n")




-----Training Models for Cluster 2-----
Training Complete for Cluster 2


-----Training Models for Cluster 0-----
Training Complete for Cluster 0


-----Training Models for Cluster 1-----
Training Complete for Cluster 1




In [ ]:
regression_results

{np.int32(2): {'Ridge': {'model': Ridge(alpha=10),
   'RMSE': 3200.8281800840596,
   'R2': 0.9347511461178278},
  'Lasso': {'model': Lasso(),
   'RMSE': 3200.8558364385767,
   'R2': 0.9347500185638766}},
 np.int32(0): {'Ridge': {'model': Ridge(alpha=10),
   'RMSE': 2757.4445278888893,
   'R2': 0.9522187888515746},
  'Lasso': {'model': Lasso(),
   'RMSE': 2757.468358403865,
   'R2': 0.9522179629740083}},
 np.int32(1): {'Ridge': {'model': Ridge(alpha=10),
   'RMSE': 8816.669493123107,
   'R2': 0.25763726713395974},
  'Lasso': {'model': Lasso(),
   'RMSE': 8816.672036644964,
   'R2': 0.2576368388053011}}}

# LightGBM on Clusters

In [ ]:
lightgbm_results = {}
for cluster in df["cluster"].unique():
  X = df[df["cluster"] == cluster].drop(['2024_price', "cluster"], axis=1)
  y = df[df["cluster"] == cluster]['2024_price']


  X_numerical = X[numerical_cols]
  scaler = StandardScaler()
  scaled_features = scaler.fit_transform(X_numerical)

  label_encoders = {}
  for col in categorical_cols:
      le = LabelEncoder()
      X[col] = le.fit_transform(X[col].astype(str))
      label_encoders[col] = le

  X_combined = np.hstack((X[categorical_cols].values, scaled_features))

  X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)


  model = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)  # Adjust hyperparameters as needed
  print(f"-----Training LightGBM for cluster {cluster}-----")
  model.fit(X_train, y_train)


  y_pred = model.predict(X_test)
  rmse = root_mean_squared_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)
  lightgbm_results[cluster] = {"model": model, "RMSE": rmse, "R2": r2}
  print("Training Complete")
  print(f"Root Mean Squared Error: {rmse}")
  print(f"R-squared: {r2}\n\n")



-----Training LightGBM for cluster 2-----
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001859 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1190
[LightGBM] [Info] Number of data points in the train set: 56952, number of used features: 10
[LightGBM] [Info] Start training from score 16883.227155


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Training Complete
Root Mean Squared Error: 2186.8120110103027
R-squared: 0.9701892772492449


-----Training LightGBM for cluster 0-----
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017333 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1154
[LightGBM] [Info] Number of data points in the train set: 109297, number of used features: 10
[LightGBM] [Info] Start training from score 33555.216547


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Training Complete
Root Mean Squared Error: 2739.100704035502
R-squared: 0.9540821124453215


-----Training LightGBM for cluster 1-----


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.036392 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1391
[LightGBM] [Info] Number of data points in the train set: 368864, number of used features: 10
[LightGBM] [Info] Start training from score 12572.237023


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Training Complete
Root Mean Squared Error: 5959.27681733395
R-squared: 0.6634219784493796




# Deep Learning (Abandoned for Simpler Methods)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
class CarPricePredictor(nn.Module):
  def __init__(self, input_size):
    super(CarPricePredictor, self).__init__()
    self.fc1 = nn.Linear(input_size, 128)
    self.fc2 = nn.Linear(128, 64)
    self.fc3 = nn.Linear(64, 1)

    self.nonlin = nn.ReLU()

  def forward(self, x):
    x = self.nonlin(self.fc1(x))
    x = self.nonlin(self.fc2(x))
    x = self.fc3(x)
    return x


In [ ]:
def rmse_loss(y_true, y_pred):
  return torch.sqrt(torch.mean((y_true - y_pred)**2))


In [ ]:
def train_model(X_train, y_train, X_test, y_test, lr, epochs, batch_size, input_size):
  X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
  y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
  y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

  train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

  model = CarPricePredictor(input_size)
  criterion = rmse_loss
  optimizer = optim.Adam(model.parameters(), lr= lr)

  for epoch in range(epochs):
    print(f"Epoch {epoch}")
    model.train()
    epoch_loss = 0
    for batch_X, batch_y in train_loader:
      optimizer.zero_grad()
      pred = model(batch_X)
      loss = criterion(batch_y, pred)
      loss.backward()
      optimizer.step()
      epoch_loss += loss.item()


    print(f"Epoch {epoch}, Loss: {epoch_loss}")

    model.eval()
    with torch.no_grad():
        test_predictions = model(X_test_tensor)
        test_rmse = criterion(test_predictions, y_test_tensor).item()

    print(f"Test RMSE: {test_rmse:.4f}")
  return model


In [ ]:
# Filter data for cluster 1
X = df[df["cluster"] == 1].drop(columns=['2024_price'])  # Drop target column
y = df[df["cluster"] == 1]['2024_price'].values  # Convert to NumPy array


categorical_cols = ['make', 'model', 'trim', 'state', 'color', 'interior']
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le


numerical_cols = ['year', 'condition', 'odometer', 'mmr', 'sale_year']

# Convert to NumPy arrays
X_numerical = X[numerical_cols].values
X_categorical = X[categorical_cols].values


X_combined = np.hstack((X_categorical, X_numerical))


X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Ensure all data is NumPy before passing to train_model
X_train = np.array(X_train, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)

# Train model
input_size = X_train.shape[1]  # Number of features
model = train_model(X_train, y_train, X_test, y_test, lr=0.01, epochs=50, batch_size=32, input_size=input_size)

Epoch 0
Epoch 0, Loss: 119094681.7734375
Test RMSE: 21795.7930
Epoch 1
Epoch 1, Loss: 62013609.6484375
Test RMSE: 21777.4082
Epoch 2
Epoch 2, Loss: 61946367.236328125
Test RMSE: 21759.1621
Epoch 3
Epoch 3, Loss: 61868199.47705078
Test RMSE: 21740.9746
Epoch 4
Epoch 4, Loss: 61834712.08300781
Test RMSE: 21722.7988
Epoch 5
Epoch 5, Loss: 61730813.41455078
Test RMSE: 21704.6445
Epoch 6
Epoch 6, Loss: 61732150.302246094
Test RMSE: 21686.5332
Epoch 7
Epoch 7, Loss: 61671884.5078125
Test RMSE: 21668.4551
Epoch 8
Epoch 8, Loss: 61638710.03417969
Test RMSE: 21650.3945
Epoch 9
Epoch 9, Loss: 61481923.829589844
Test RMSE: 21632.3555
Epoch 10
Epoch 10, Loss: 61394805.53613281
Test RMSE: 21614.3379
Epoch 11
Epoch 11, Loss: 61366397.951171875
Test RMSE: 21596.3496
Epoch 12
Epoch 12, Loss: 61333066.13330078
Test RMSE: 21578.3750
Epoch 13
Epoch 13, Loss: 61269964.92285156
Test RMSE: 21560.4785
Epoch 14
Epoch 14, Loss: 61222385.078125
Test RMSE: 21542.5586
Epoch 15
Epoch 15, Loss: 61148110.114746094
T

# KFold CV on Column 1

In [ ]:
X = df[df["cluster"] == 1].drop('2024_price', axis=1)
y = df[df["cluster"] == 1]['2024_price']

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le

X_numerical = X[numerical_cols].values
X_combined = np.hstack((X[categorical_cols].values, X_numerical))


kf = KFold(n_splits= 5, shuffle= True)

# X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

model = lgb.LGBMRegressor(num_leaves= 50, n_estimators=50, learning_rate=0.5, random_state=42)
# model.fit(X_train, y_train)


# y_pred = model.predict(X_test)
# rmse = root_mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# print(f"Root Mean Squared Error: {rmse}")
# print(f"R-squared: {r2}")


scorer = make_scorer(mean_squared_error)

scores = cross_val_score(model, X_combined, y, cv=kf, scoring=scorer, n_jobs=-1)

# Print results
print(f"Mean RMSE: {np.sqrt(np.mean(scores)):.4f}, Std Dev: {np.std(scores):.4f}")

Mean RMSE: 5738.9168, Std Dev: 1572295.3991
